In [25]:
import numpy as np
from selenium import webdriver
from time import sleep
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
import random
import threading
import concurrent.futures

In [26]:
# Customize Chrome browser settings
chrome_options = Options()                                     
chrome_options.add_argument('--no-sandbox')                     # Them tuy chon de chay Chrome ko co sandbox, thuong duoc su dung de tranh cac van de ve quyen truy cap he thong
chrome_options.add_argument('--disable-notifications')          # Tat thong bao tu trinh duyet
chrome_options.add_argument('--disable-infobars')               # Tat thanh thong tin cua trinh duyet
#chrome_options.add_argument('--headless')

https://www.lazada.vn/catalog/?page=1&q=quần+áo

In [27]:
def getDataIn1Page(search_, page_, data_lock, data_crawled):
    base_link = 'https://www.lazada.vn/catalog/?page={page}&q={search}'
    link = base_link.format(page = page_, search = search_)
    # Access
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver.get(link)
    driver.implicitly_wait(2)

    # Get link/title
    elems = driver.find_elements(By.CSS_SELECTOR , ".RfADt [href]")
    titles = [elem.text for elem in elems]
    links = [elem.get_attribute('href') for elem in elems]

    # Get type
    type_name = [elem.get_attribute("value") for elem in driver.find_elements(By.CSS_SELECTOR, ".search-box__input--O34g")]
    type_name = type_name * len(titles)

    # Get price sale
    elems_price_sale = driver.find_elements(By.CSS_SELECTOR , ".aBrP0")
    prices = [elem_price.text for elem_price in elems_price_sale]

    # Get discount, sold, preview
    discount_percent_list, sold_list, preview_list = [], [], []
    for i in range(1, len(titles)+1):
        try:
            discount_percent = driver.find_element("xpath", "/html/body/div[3]/div/div[2]/div[1]/div/div[1]/div[2]/div[{}]/div/div/div[2]/div[4]/span".format(i))
            discount_percent_list.append(discount_percent.text)
        except NoSuchElementException:
            discount_percent_list.append(None)

        try:
            sold = driver.find_element("xpath", "/html/body/div[3]/div/div[2]/div[1]/div/div[1]/div[2]/div[{}]/div/div/div[2]/div[5]/span[1]/span[1]".format(i))
            sold_list.append(sold.text)
        except NoSuchElementException:
            sold_list.append(None)

        try:
            preview = driver.find_element("xpath", "/html/body/div[3]/div/div[2]/div[1]/div/div[1]/div[2]/div[{}]/div/div/div[2]/div[5]/div/span".format(i))
            preview_list.append(preview.text)
        except NoSuchElementException:
            preview_list.append(None)
    
    # Get Location
    elems_locations = driver.find_elements(By.CSS_SELECTOR , ".oa6ri ")
    locations = [elem.text for elem in elems_locations]

    # Cập nhật dữ liệu vào biến chung
    with data_lock:
        data_crawled['type'].extend(type_name)
        data_crawled['title'].extend(titles)
        data_crawled['links'].extend(links)
        data_crawled['price'].extend(prices)
        data_crawled['discount_percent'].extend(discount_percent_list)
        data_crawled['sold'].extend(sold_list)
        data_crawled['preview'].extend(preview_list)
        data_crawled['location'].extend(locations)
    
    driver.quit()

In [28]:
dataProducts = {
    "type": [],
    "title": [],
    "links": [],
    "price": [],
    "discount_percent": [],
    "sold": [],
    "preview": [],
    "location": []
}

In [29]:
def crawlDataWithMultiplesThread(product, num_pages, num_threads_per_loop):
    # Lock để đảm bảo thread-safe khi cập nhật dữ liệu
    data_lock = threading.Lock()
    
    # Lặp qua các trang và chạy các luồng
    for start_page in tqdm(range(1, num_pages + 1, num_threads_per_loop), desc="Crawling Data"):
        threads = []
        for page in range(start_page, start_page + num_threads_per_loop):
            thread = threading.Thread(target=getDataIn1Page, args=(product, page, data_lock, dataProducts))
            threads.append(thread)
            thread.start()

        # Đợi tất cả các luồng hoàn thành và thu thập dữ liệu đã crawl
        for thread in threads:
            thread.join()

        print(f"Crawling completed from pages {start_page} to {min(start_page + num_threads_per_loop - 1, num_pages)}")

    print("Crawling completed.")

    return dataProducts

In [30]:
dataProducts = crawlDataWithMultiplesThread("quần+áo", 1, 1)

Crawling Data: 100%|██████████| 1/1 [00:56<00:00, 56.51s/it]

Crawling completed from pages 1 to 1
Crawling completed.


In [31]:
dataProducts

{'type': ['quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo',
  'quần áo'],
 'title': ['Set Baggy Jeans Ống Suông Đai Nút Rách + Sơ mi rút dâY HOT',
  'Đồ Bộ Nữ Đi Chơi Chất Thun Mát Size M - dưới 53kg / Size L Từ 54kg đến 65kg/ Size XL từ 65kg đến 72kg',
  'Bộ Pijama, Đồ Bộ Mặc Nhà Nữ Dáng Cộc xinh xắn',
  'Đồ ngủ nữ pijama mặc nhà bộ ngủ hoạt hình cộc tay quần đùi mùa hè dễ thương',
  'Đồ bộ nữ mặc nhà dễ thương , bộ ngủ pijama ngắn hoạt hình cộc tay mùa hè',
  'Bộ Đồ Ngủ Mỏng Mùa Hè Tay Ngắn Phong Cách Mới 2023 Cho Nữ [Lff0707]',
  '[Đủ s

In [32]:
df = pd.DataFrame(dataProducts)

In [33]:
df

,type,title,links,price,discount_percent,sold,preview,location
0,quần áo,Set Baggy Jeans Ống Suông Đai Nút Rách + Sơ mi...,https://www.lazada.vn/products/set-baggy-jeans...,149.000 ₫,None,5 Đã bán,None,Hồ Chí Minh
1,quần áo,Đồ Bộ Nữ Đi Chơi Chất Thun Mát Size M - dưới 5...,https://www.lazada.vn/products/do-bo-nu-di-cho...,35.891 ₫,64% Off,3.0K Đã bán,(1013),Hồ Chí Minh
2,quần áo,"Bộ Pijama, Đồ Bộ Mặc Nhà Nữ Dáng Cộc xinh xắn",https://www.lazada.vn/products/bo-pijama-do-bo...,15.971 ₫,54% Off,229 Đã bán,(65),Hà Nội
3,quần áo,Đồ ngủ nữ pijama mặc nhà bộ ngủ hoạt hình cộc ...,https://www.lazada.vn/products/do-ngu-nu-pijam...,25.000 ₫,50% Off,902 Đã bán,(241),Hồ Chí Minh
4,quần áo,"Đồ bộ nữ mặc nhà dễ thương , bộ ngủ pijama ngắ...",https://www.lazada.vn/products/do-bo-nu-mac-nh...,25.000 ₫,50% Off,241 Đã bán,(54),Hồ Chí Minh
5,quần áo,Bộ Đồ Ngủ Mỏng Mùa Hè Tay Ngắn Phong Cách Mới ...,https://www.lazada.vn/products/bo-do-ngu-mong-...,27.200 ₫,26% Off,None,None,Nước ngoài
6,quần áo,[Đủ size 35-55 cân] Sét bộ 3 món áo 2 dây+áo c...,https://www.lazada.vn/products/du-size-35-55-c...,79.000 ₫,43% Off,960 Đã bán,(259),Hà Nội
7,quần áo,Bộ mẫu hoạ tiết Sư Thích Minh Tuệ quần đùi tay...,https://www.lazada.vn/products/bo-mau-hoa-tiet...,37.000 ₫,8% Off,78 Đã bán,(15),Cần Thơ
8,quần áo,[3 SIZE 45-70KG) Đồ bộ LỤA ĐÙI CHẤM BI CHẤT VẢ...,https://www.lazada.vn/products/3-size-45-70kg-...,33.454 ₫,66% Off,6.0K Đã bán,(1724),Hồ Chí Minh
9,quần áo,Bộ Đồ Ngủ Tay Ngắn Dáng Rộng In Họa Tiết Hoạt ...,https://www.lazada.vn/products/bo-do-ngu-tay-n...,41.600 ₫,54% Off,24 Đã bán,(6),Nước ngoài


In [34]:
# Hàm cuộn từ từ xuống dưới cùng của trang cho đến khi get element được thì dừng
def getStarItem(driver, step=200, wait_time=0.1):
    last_height = driver.execute_script("return document.body.scrollHeight")
    current_position = 0
    
    while current_position < last_height:
        driver.execute_script(f"window.scrollBy(0, {step});")
        sleep(wait_time)
        current_position += step
        elems_stars = driver.find_elements(By.CSS_SELECTOR, ".percent")
        if not (elems_stars == []):
            break
    if not (elems_stars == []):
        return [elem.text for elem in elems_stars]
    else :
        return ['N/A'] * 5

In [35]:
def getDetailItems(link):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver.get(link)
    driver.implicitly_wait(1)
    # get original price
    try:
        elems_original = driver.find_element("xpath", "/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[10]/div/div/div/span[1]")
        original = elems_original.text
    except NoSuchElementException:
        original = None
    # get ship price
    try:
        elems_ship_price = driver.find_element("xpath", "/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[1]/div/div/div[3]/div/div[1]/div/div[1]/div[2]")
        ship_price = elems_ship_price.text
    except NoSuchElementException:
        ship_price = None
    # get return_
    try:
        elems_return = driver.find_element("xpath", "/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[3]/div/div[2]/div[2]/div/div/div/div")
        return_ = elems_return.text
    except NoSuchElementException:
        return_ = None
    # get positive review
    try:
        elems_positive_review = driver.find_element("xpath", "/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[6]/div/div[2]/div[1]/div[2]")
        positive_review = elems_positive_review.text
    except NoSuchElementException:
        positive_review = None
    # get ship on time
    try:
        elems_ship_on_time = driver.find_element("xpath", "/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[6]/div/div[2]/div[2]/div[2]")
        ship_on_time = elems_ship_on_time.text
    except NoSuchElementException:
        ship_on_time = None
    # get rate response
    try:
        elems_rate_response = driver.find_element("xpath", "/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[6]/div/div[2]/div[3]/div[2]")
        rate_response = elems_rate_response.text
    except NoSuchElementException:
        rate_response = None
    # get stars
    stars = getStarItem(driver)
    star_5 = stars[0]
    star_4 = stars[1]
    star_3 = stars[2]
    star_2 = stars[3]
    star_1 = stars[4]
    # into a record
    # into a record
    data = {
        "original": original,
        "ship_price": ship_price,
        "return": return_,
        "positive_review": positive_review,
        "ship_on_time": ship_on_time,
        "rate_response": rate_response,
        "one_star": star_1,
        "two_star": star_2,
        "three_star": star_3,
        "four_star": star_4,
        "five_star": star_5
    }

    driver.quit()
    return data

In [36]:
detailProducts = {
    "original": [],
    "ship_price": [],
    "return": [],
    "positive_review": [],
    "ship_on_time": [],
    "rate_response": [],
    "one_star": [],
    "two_star": [],
    "three_star": [],
    "four_star": [],
    "five_star": []
}

In [37]:
# Sử dụng ThreadPoolExecutor để chạy đa luồng
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    # Sử dụng list comprehension để gọi hàm getDetailItems cho từng link trong df['links']
    results = list(tqdm(executor.map(getDetailItems, df['links']), total=len(df['links']), desc="Crawling product details"))

Crawling product details:   2%|▎         | 1/40 [00:51<33:14, 51.14s/it]


In [1]:
# Create a progress bar to track the crawl process
for link in tqdm(df.links, desc="Crawling product details"):
    details = getDetailItems(link)
    for key in detailProducts.keys():
        detailProducts[key].append(details[key])

NameError: name 'tqdm' is not defined

In [ ]:
# add columns to df
for key, value in detailProducts.items():
    df[key] = value

In [ ]:
# save to csv
df.to_csv("lazada.csv", index = True)
df.head()

,type,title,links,price,discount_percent,sold,preview,location,original,ship_price,return,positive_review,ship_on_time,rate_response,one_star,two_star,three_star,four_star,five_star
0,[Áo thun nữ],[Set Baggy Jeans Ống Suông Đai Nút Rách + Sơ m...,[https://www.lazada.vn/products/set-baggy-jean...,[149.000 ₫],[None],[5 Đã bán],[None],[Hồ Chí Minh],None,18.300 ₫,3 ngày trả hàng cho Nhà bán hàng,92%,99%,63%,0,0,0,0,0
1,[Áo thun nữ],"[Bộ Pijama, Đồ Bộ Mặc Nhà Nữ Dáng Cộc xinh xắn]",[https://www.lazada.vn/products/bo-pijama-do-b...,[15.971 ₫],[54% Off],[229 Đã bán],[(63)],[Hà Nội],59.000 ₫,32.700 ₫,3 ngày trả hàng cho Nhà bán hàng,95%,100%,100%,0,1,0,3,59
2,[Áo thun nữ],[Đồ Bộ Nữ Đi Chơi Chất Thun Mát Size M - dưới ...,[https://www.lazada.vn/products/do-bo-nu-di-ch...,[35.891 ₫],[64% Off],[3.0K Đã bán],[(1011)],[Hồ Chí Minh],100.000 ₫,16.500 ₫,3 ngày trả hàng cho Nhà bán hàng,97%,100%,100%,20,7,27,35,922
3,[Áo thun nữ],[Bộ mẫu hoạ tiết Sư Thích Minh Tuệ quần đùi ta...,[https://www.lazada.vn/products/bo-mau-hoa-tie...,[37.000 ₫],[8% Off],[78 Đã bán],[(9)],[Cần Thơ],45.000 ₫,28.700 ₫,3 ngày trả hàng cho Nhà bán hàng,96%,100%,60%,0,0,0,0,10
4,[Áo thun nữ],"[Đồ bộ nữ mặc nhà dễ thương , bộ ngủ pijama ng...",[https://www.lazada.vn/products/do-bo-nu-mac-n...,[25.000 ₫],[50% Off],[241 Đã bán],[(54)],[Hồ Chí Minh],75.999 ₫,16.500 ₫,3 ngày trả hàng cho Nhà bán hàng,91%,100%,57%,1,1,2,2,48
